In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

image_size = (64, 64)
batch_size = 64
epochs = 30

train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=40,
    zoom_range=0.4,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)
val_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    r"C:\Users\surej\OneDrive\Desktop\project\assignment project\New Plant Diseases Dataset(Augmented)\New Plant Diseases Dataset(Augmented)\train",
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)
val_generator = val_datagen.flow_from_directory(
    r"C:\Users\surej\OneDrive\Desktop\project\assignment project\New Plant Diseases Dataset(Augmented)\New Plant Diseases Dataset(Augmented)\valid",
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.3),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.35),

    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.4),

    GlobalAveragePooling2D(),

    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3, verbose=1)

model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs,
    callbacks=[early_stopping, reduce_lr]
)

model.save('plant_disease_model_v2.h5')
print("Model saved as 'plant_disease_model_v2.h5'.")

loss, accuracy = model.evaluate(val_generator)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")


Found 69858 images belonging to 38 classes.
Found 17474 images belonging to 38 classes.
Epoch 1/30
1092/1092 ━━━━━━━━━━━━━━━━━━━━ 6920s 6s/step - accuracy: 0.1592 - loss: 3.1958 - val_accuracy: 0.3898 - val_loss: 2.1506 - learning_rate: 0.0010
Epoch 2/30
1092/1092 ━━━━━━━━━━━━━━━━━━━━ 6244s 6s/step - accuracy: 0.4339 - loss: 1.9125 - val_accuracy: 0.3909 - val_loss: 2.3225 - learning_rate: 0.0010
Epoch 3/30
1092/1092 ━━━━━━━━━━━━━━━━━━━━ 6690s 6s/step - accuracy: 0.5380 - loss: 1.5267 - val_accuracy: 0.4753 - val_loss: 2.0795 - learning_rate: 0.0010
Epoch 4/30
1092/1092 ━━━━━━━━━━━━━━━━━━━━ 7514s 7s/step - accuracy: 0.6144 - loss: 1.2873 - val_accuracy: 0.4592 - val_loss: 2.5748 - learning_rate: 0.0010
Epoch 5/30
1092/1092 ━━━━━━━━━━━━━━━━━━━━ 6961s 6s/step - accuracy: 0.6620 - loss: 1.1322 - val_accuracy: 0.4468 - val_loss: 2.6623 - learning_rate: 0.0010
Epoch 6/30
1092/1092 ━━━━━━━━━━━━━━━━━━━━ 7893s 7s/step - accuracy: 0.6923 - loss: 1.0361 - val_accuracy: 0.6281 - val_loss: 1.5865 

Model saved as 'plant_disease_model_v2.h5'.
274/274 ━━━━━━━━━━━━━━━━━━━━ 31s 115ms/step - accuracy: 0.8722 - loss: 0.4460
Validation Loss: 0.4416092038154602
Validation Accuracy: 0.8712372779846191


In [ ]:
%%writefile disease.py
import streamlit as st
import numpy as np
import tensorflow as tf
from PIL import Image

model = tf.keras.models.load_model('plant_disease_model_v2.h5')

class_names = [
    'Apple___Apple_scab',
    'Apple___Black_rot',
    'Apple___Cedar_apple_rust',
    'Apple___healthy',
    'Blueberry___healthy',
    'Cherry_(including_sour)___Powdery_mildew',
    'Cherry_(including_sour)___healthy',
    'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
    'Corn_(maize)___Common_rust',
    'Corn_(maize)___Northern_Leaf_Blight',
    'Corn_(maize)___healthy',
    'Grape___Black_rot',
    'Grape___Esca_(Black_Measles)',
    'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
    'Grape___healthy',
    'Orange___Haunglongbing_(Citrus_greening)',
    'Peach___Bacterial_spot',
    'Peach___healthy',
    'Pepper,_bell___Bacterial_spot',
    'Pepper,_bell___healthy',
    'Potato___Early_blight',
    'Potato___Late_blight',
    'Potato___healthy',
    'Raspberry___healthy',
    'Soybean___healthy',
    'Squash___Powdery_mildew',
    'Strawberry___Leaf_scorch',
    'Strawberry___healthy',
    'Tomato___Bacterial_spot',
    'Tomato___Early_blight',
    'Tomato___Late_blight',
    'Tomato___Leaf_Mold',
    'Tomato___Septoria_leaf_spot',
    'Tomato___Spider_mites Two-spotted_spider_mite',
    'Tomato___targetspot',
    'Tomato___Yellow_Leaf_Curl_Virus',
    'Tomato___Tomato_mosaic_virus',
    'Tomato___healthy'
]

st.title("Plant Disease Detection")
st.write("Upload an image of a plant leaf, and this app will predict the disease.")

uploaded_file = st.file_uploader("Choose an image file", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption="Uploaded Image", use_column_width=True)
    
    def preprocess_image(image):
        image = image.resize((64, 64))  
        image = np.array(image) / 255.0  
        image = np.expand_dims(image, axis=0)  
        return image
    
    processed_image = preprocess_image(image)
    
    prediction = model.predict(processed_image)
    predicted_class = class_names[np.argmax(prediction)]  
    confidence = np.max(prediction) * 100  
    
    st.write(f"**Prediction:** {predicted_class}")
    st.write(f"**Confidence:** {confidence:.2f}%")


Overwriting disease.py


In [38]:
!streamlit run disease.py

^C
